This notebook performs data cleaning and preprocessing for each dataset listed in the configuration file. It removes unnecessary columns, handles missing values, and saves the clean DataFrame in an efficient format in the processed data folder.

### Configuration and Library Imports

In [1]:
%run ../src/settings.py

In [ ]:
config = load_config()

# Define processing parameters from config
raw_data_path = config['data']['raw_path']
processed_data_path = config['data']['processed_path']
cols_to_drop = ["RendaMedia"]
id_column = "Cod_Setor"

In [3]:
config = load_config()

# Define os parâmetros de processamento a partir do config
raw_data_path = config['data']['raw_path']
processed_data_path = config['data']['processed_path']
cols_to_drop = ["RendaMedia"]
id_column = "Cod_Setor"

In [ ]:
processing_results = []

for file_name in config['datasets']:
  try:
    # Load raw data
    df_raw = load_raw_data(filename=file_name, data_path=raw_data_path)

    # Prepare data (cleaning and feature selection)
    df_processed = prepare_data(df_raw, cols_to_drop=cols_to_drop)

    # Define output file name
    output_file_name = f"{file_name.split('.')[0]}.parquet"

    # Save result
    res = save_processed_data(df_processed, filename=output_file_name,
                              data_path=processed_data_path)

    processing_results.append(
      {'file_name': file_name,
       'file_processed': res,
       'total_rows': len(df_processed),
       'status': 'success',
       'error': None}
    )

  except Exception as e:
    processing_results.append(
      {'file_name': file_name,
       'file_processed': None,
       'total_rows': 0,
       'status': 'error',
       'error': str(e)}
    )

df_processing_results = pd.DataFrame(processing_results)
display(df_processing_results)